In [3]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import sys
import numpy as np
import pickle

sys.path.append('D:\\Program\\MyCode\\Round_robin_SL\\Round-Robin')
from models import *
from clients_datasets import *
from tqdm.notebook import tqdm
from utils import *
from AttFunc import *
from Fisher_LeNet import *

In [4]:
batch_size = 600
epochs = 30
NC = 10
dataset = 'mnist'

clients_trainloader = load_clients_trainsets(dataset, NC, batch_size)
clients_testloader = load_clients_testsets(dataset, NC, batch_size)

server, server_opt, clients, clients_opts = set_model_and_opt(dataset, NC)
client_level = 1
server_level = 4

criterion = torch.nn.CrossEntropyLoss()

In [5]:
clients_acc0 = []
#################################################################################
###########################   Normal training   #################################
# train
server.train()
for i in range(NC):
    clients[i].train()
server.apply(init_weights)
clients[0].apply(init_weights)
last_trained_params = clients[0].state_dict()
for epoch in tqdm(range(epochs), desc="Normal Training", unit="eopch"):
# for epoch in range(epochs):
    for idx, client in enumerate(clients):
        client.load_state_dict(last_trained_params)
        for j, data in enumerate(clients_trainloader[idx]):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            smashed_data = client.forward(images, client_level=client_level)
            output = server.forward(smashed_data, server_level=server_level)
            clients_opts[idx].zero_grad()
            server_opt.zero_grad()
            loss = criterion(output, labels)
            loss.backward()
            clients_opts[idx].step()
            server_opt.step()
        last_trained_params = client.state_dict()
for i in range(NC):
    clients[i].load_state_dict(last_trained_params)
# test
server.eval()
for i in range(NC):
    clients[i].eval()
with torch.no_grad():
    for idx, client in enumerate(tqdm(clients)):
        correct = 0
        total = 0
        acc0 = 0
        for data in clients_testloader[idx]:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()

            smashed_data = client.forward(images, client_level=client_level)
            output = server.forward(smashed_data, server_level=server_level)
            _, pre = torch.max(output.data, 1)
            total += images.shape[0]
            correct += (pre == labels).sum().item()
        acc0 = 100 * correct / total
        clients_acc0.append(acc0)
acc0 = np.mean(clients_acc0)
print(acc0)

Normal Training:   0%|          | 0/30 [00:00<?, ?eopch/s]

  0%|          | 0/10 [00:00<?, ?it/s]

98.82000000000001


In [14]:
att_type = 'sign'
acc0 = 98.88
am = LeNet_5().cuda()
am_train(am, clients_trainloader[8], dataset)
betas = []

for i in range(10):
    beta = simulated_annealing(dataset, 1, 500, att_type, acc0,  clients_testloader[8])
    betas.append(beta)
    print(beta)

Training attack model:   0%|          | 0/100 [00:00<?, ?eopch/s]

1.8883856151707787
1.0524798494931977
0.48120772444317617
0.8940559062514558
1.1325366357722637
0.02642600490295226
1.3815886638871187
0.6363913235183403
0.7018389063531633
0.17936669931408938


In [15]:
beta_mean, beta_range = data_process(betas)
print(beta_mean)

0.8211914964040392


-----------------------
## without fisher

In [16]:
acc0 = 98.88
att_type = 'sign'
acc1_varying = []
drop_varying = []
clients_acc1 = []
clients_drop = []
iters = 5


for iter in tqdm(range(iters), desc="Training", unit="iter"):
    batch_size = 600
    epochs = 30
    NC = 10
    dataset = 'mnist'

    clients_trainloader = load_clients_trainsets(dataset, NC, batch_size)
    clients_testloader = load_clients_testsets(dataset, NC, batch_size)

    server, server_opt, clients, clients_opts = set_model_and_opt(dataset, NC)
    client_level = 1
    server_level = 4

    criterion = torch.nn.CrossEntropyLoss()
    # train
    mal_client_id = [8]
    server.train()
    for i in range(NC):
        clients[i].train()
    server.apply(init_weights)
    clients[0].apply(init_weights)
    last_trained_params = clients[0].state_dict()
    for epoch in range(epochs):
        beta = beta_mean
        for idx, client in enumerate(clients):
            client.load_state_dict(last_trained_params)
            for j, data in enumerate(clients_trainloader[idx]):
                # training part
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()
                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                clients_opts[idx].zero_grad()
                server_opt.zero_grad()
                loss = criterion(output, labels)
                loss.backward()
                clients_opts[idx].step()
                server_opt.step()
            # weight sharing
            last_trained_params = client.state_dict()
            # attack part
            if idx in mal_client_id :
                benign_params = list(client.parameters())[:2]
                mal_params = perturbation(beta=beta, bp=benign_params, type=att_type)
                last_trained_params['conv1.0.weight'] = mal_params[0]
                last_trained_params['conv1.0.bias'] = mal_params[1]
    for i in range(NC):
        clients[i].load_state_dict(last_trained_params)

    # test
    server.eval()
    for i in range(NC):
        clients[i].eval()
    with torch.no_grad():
        for idx, client in enumerate(clients):
            correct = 0
            total = 0
            acc1 = 0
            for data in clients_testloader[idx]:
                images, labels = data
                images, labels = images.cuda(), labels.cuda()

                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                _, pre = torch.max(output.data, 1)
                total += images.shape[0]
                correct += (pre == labels).sum().item()
            acc1 = 100 * correct / total
            clients_acc1.append(acc1)
            drop = acc0 - acc1
            clients_drop.append(drop)
    acc1 = np.mean(clients_acc1)
    acc1_varying.append(acc1)
    drop = np.mean(clients_drop)
    drop_varying.append(drop)

Training:   0%|          | 0/5 [00:00<?, ?iter/s]

In [17]:
print(acc1_varying)
print(drop_varying)

[97.35999999999999, 97.305, 97.39999999999999, 97.3475, 97.31800000000001]
[1.5199999999999931, 1.5749999999999944, 1.4799999999999949, 1.5324999999999949, 1.5619999999999954]


In [18]:
acc1_mean, acc1_range = data_process(acc1_varying)
print('acc1:')
print(acc1_mean)
print(acc1_range)
print('---------------------')
drop_mean, drop_range = data_process(drop_varying)
print('drop:')
print(drop_mean)
print(drop_range)

acc1:
97.35374999999999
0.012499999999988631
---------------------
drop:
1.526249999999994
0.012500000000001732


-----------------------------------
## with fisher

In [19]:
acc0 = 98.88
att_type = 'sign'
acc1_varying = []
drop_varying = []
clients_acc1 = []
clients_drop = []
iters = 5


for iter in tqdm(range(iters), desc="Training", unit="iter"):
    batch_size = 600
    epochs = 30
    NC = 10
    dataset = 'mnist'

    clients_trainloader = load_clients_trainsets(dataset, NC, batch_size)
    clients_testloader = load_clients_testsets(dataset, NC, batch_size)

    server, server_opt, clients, clients_opts = set_model_and_opt(dataset, NC)
    client_level = 1
    server_level = 4

    criterion = torch.nn.CrossEntropyLoss()
    # train
    mal_client_id = [8]
    server.train()
    for i in range(NC):
        clients[i].train()
    server.apply(init_weights)
    clients[0].apply(init_weights)
    last_trained_params = clients[0].state_dict()
    for epoch in range(epochs):
        beta = beta_mean
        for idx, client in enumerate(clients):
            client.load_state_dict(last_trained_params)
            for j, data in enumerate(clients_trainloader[idx]):
                # training part
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()
                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                clients_opts[idx].zero_grad()
                server_opt.zero_grad()
                loss = criterion(output, labels)
                loss.backward()
                clients_opts[idx].step()
                server_opt.step()
            # weight sharing
            last_trained_params = client.state_dict()
            # attack part
            if idx in mal_client_id :
                benign_params = list(client.parameters())[:2]

                fisher_matrix = {}
                for param_name, param in client.named_parameters():
                    if param_name == 'conv1.0.weight':
                        grad = param.grad.cpu().detach().numpy()
                        if param_name not in fisher_matrix:
                            fisher_matrix[param_name] = grad ** 2
                        else:
                            fisher_matrix[param_name] += grad ** 2
                    if param_name == 'conv1.0.bias':
                        grad = param.grad.cpu().detach().numpy()
                        if param_name not in fisher_matrix:
                            fisher_matrix[param_name] = grad ** 2
                        else:
                            fisher_matrix[param_name] += grad ** 2
                weight_positions = []
                bias_positions = []
                weight_positions.append(find_positions(fisher_matrix['conv1.0.weight'], 0.333))
                bias_positions.append(find_positions(fisher_matrix['conv1.0.bias'], 0.333))

                mal_params = fisher_perturbation(client_level, beta, benign_params, weight_positions, bias_positions, type=att_type)
                last_trained_params['conv1.0.weight'] = mal_params[0]
                last_trained_params['conv1.0.bias'] = mal_params[1]
    for i in range(NC):
        clients[i].load_state_dict(last_trained_params)

    # test
    server.eval()
    for i in range(NC):
        clients[i].eval()
    with torch.no_grad():
        for idx, client in enumerate(clients):
            correct = 0
            total = 0
            acc1 = 0
            for data in clients_testloader[idx]:
                images, labels = data
                images, labels = images.cuda(), labels.cuda()

                smashed_data = client.forward(images, client_level=client_level)
                output = server.forward(smashed_data, server_level=server_level)
                _, pre = torch.max(output.data, 1)
                total += images.shape[0]
                correct += (pre == labels).sum().item()
            acc1 = 100 * correct / total
            clients_acc1.append(acc1)
            drop = acc0 - acc1
            clients_drop.append(drop)
    acc1 = np.mean(clients_acc1)
    acc1_varying.append(acc1)
    drop = np.mean(clients_drop)
    drop_varying.append(drop)

Training:   0%|          | 0/5 [00:00<?, ?iter/s]

In [20]:
print(acc1_varying)
print(drop_varying)

[94.9, 94.50000000000001, 88.94333333333333, 89.9075, 90.72200000000001]
[3.979999999999997, 4.3799999999999955, 9.936666666666664, 8.972499999999997, 8.157999999999996]


In [21]:
acc1_mean, acc1_range = data_process(acc1_varying)
print('acc1:')
print(acc1_mean)
print(acc1_range)
print('---------------------')
drop_mean, drop_range = data_process(drop_varying)
print('drop:')
print(drop_mean)
print(drop_range)

acc1:
90.72200000000001
0.0
---------------------
drop:
8.157999999999996
0.0
